In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# Import the data
PATH_DATA = "../Model-test/Data.csv"
data = pd.read_csv(PATH_DATA)
print(data.head())
no_sample = 5

     clip_name  frame_nr  person_id   nose_x   nose_y    neck_x   neck_y  \
0  1P-Ana-780p         0          0  384.510  374.092  0.901934  394.898   
1  1P-Ana-780p         1          0  384.434  377.367  0.901137  387.966   
2  1P-Ana-780p         2          0  380.987  380.877  0.900104  384.482   
3  1P-Ana-780p         3          0  377.540  380.978  0.915440  381.021   
4  1P-Ana-780p         4          0  377.356  384.518  0.906092  377.552   

   rshoulder_x  rshoulder_y  relbow_x  ...  lheal_x  lheal_y  rbigtoe_x  \
0      429.678     0.893129   321.665  ...  384.394  988.314   0.811540   
1      429.738     0.888454   321.519  ...  380.976  988.174   0.793082   
2      429.951     0.903180   321.452  ...  377.537  991.723   0.835229   
3      433.336     0.921297   318.180  ...  377.392  995.303   0.817230   
4      436.751     0.920972   314.689  ...  377.343  998.704   0.834323   

   rbigtoe_y  rsmalltoe_x  rsmalltoe_y  rheal_x  rheal_y  label  status  
0    373.889      

In [8]:
# Data preprocessing, get the input X and the label y
ind_start = data[data['status'] == "S"].index.tolist()
ind_end = data[data['status'] == "E"].index.tolist()

print(ind_start)
print(ind_end)

[101, 184, 273, 355, 446, 526, 613, 700, 789, 875]
[183, 272, 354, 445, 525, 612, 699, 788, 874, 961]


In [9]:
X = []
y = []

for i in range(no_sample):
    X.append(data.iloc[ind_start[i]: ind_end[i], 3:-2])
    print(X[i].head())
    y.append(data.loc[ind_start[i], 'label'])
    print(f'Label is {y[i]}')

      nose_x   nose_y    neck_x   neck_y  rshoulder_x  rshoulder_y  relbow_x  \
101  408.934  398.255  0.868333  415.796      496.197     0.849269   346.164   
102  405.557  391.469  0.873200  412.340      496.094     0.863101   346.068   
103  405.467  391.460  0.885386  408.929      492.767     0.884072   345.938   
104  405.457  394.854  0.894966  408.929      496.082     0.881643   345.934   
105  405.338  398.315  0.919749  408.899      496.098     0.882404   345.896   

     relbow_y  rwrist_x  rwrist_y  ...  lsmalltoe_x  lsmalltoe_y  lheal_x  \
101   499.584  0.888893   293.742  ...      880.196     0.885451  426.357   
102   496.149  0.898450   293.764  ...      887.094     0.857975  405.472   
103   496.163  0.876817   293.847  ...      897.472     0.803724  405.399   
104   496.146  0.879655   293.797  ...      897.431     0.796370  405.306   
105   496.146  0.870555   293.695  ...      897.525     0.787503  401.913   

     lheal_y  rbigtoe_x  rbigtoe_y  rsmalltoe_x  rsmallt

In [5]:
# Define hyperparameters
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 2   # number of frames
NUM_FEATURES = 50     # number of join coordinates

In [10]:
ind_samp = []

for i in range(no_sample):
    aux = np.random.randint(ind_start[i], ind_end[i], MAX_SEQ_LENGTH)
    aux.sort()
    ind_samp.append(aux) 
    
print(ind_samp)    


[array([124, 143]), array([185, 233]), array([293, 311]), array([363, 422]), array([452, 485])]


In [11]:
# Fixing format of the data
def enc_label(label):
    code = 0
    if label == "right-turn":
        code = 1
    if label == "side":
        code = 2
    if label == "cuban-basic":
        code = 3
    if label == "suzie-q":
        code = 4
    return code

In [17]:
X_train = [x.loc[ind_samp[ind], :].to_numpy() for (ind, x) in enumerate(X)]
X_train = np.array(X_train)

#X_train = X.loc[ind_samp, :].to_numpy()
print(X_train.shape)

(5, 2, 50)


In [18]:
X_train = X_train.reshape(no_sample, MAX_SEQ_LENGTH, NUM_FEATURES).astype("float32")
print(X_train.shape)

(5, 2, 50)


In [19]:
print(y)

['basic', 'basic', 'right-turn', 'right-turn', 'side']


In [29]:
#y_train = enc_label(y)
#y_train = np.array(y_train).astype("float32")

y_train = [enc_label(x) for x in y]
y_train = np.array(y_train).astype("float32")

In [30]:
print(y_train)

[0. 0. 1. 1. 2.]


In [31]:
print(type(X_train))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [39]:
model = models.Sequential()
model.add(layers.Flatten(input_shape=(MAX_SEQ_LENGTH, NUM_FEATURES)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(5, activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 100)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                6464      
                                                                 
 dense_9 (Dense)             (None, 5)                 325       
                                                                 
Total params: 6,789
Trainable params: 6,789
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [41]:
# Check the trainning accuracy
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    validation_data=(X_train, y_train)
)

Epoch 1/10
1/1 [==============================] - 1s 525ms/step - loss: 717.9631 - accuracy: 0.0000e+00 - val_loss: 565.9884 - val_accuracy: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 37ms/step - loss: 565.9884 - accuracy: 0.0000e+00 - val_loss: 421.3239 - val_accuracy: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 38ms/step - loss: 421.3239 - accuracy: 0.0000e+00 - val_loss: 358.1677 - val_accuracy: 0.2000
Epoch 4/10
1/1 [==============================] - 0s 50ms/step - loss: 358.1677 - accuracy: 0.2000 - val_loss: 310.4888 - val_accuracy: 0.4000
Epoch 5/10
1/1 [==============================] - 0s 52ms/step - loss: 310.4888 - accuracy: 0.4000 - val_loss: 253.1609 - val_accuracy: 0.4000
Epoch 6/10
1/1 [==============================] - 0s 39ms/step - loss: 253.1609 - accuracy: 0.4000 - val_loss: 186.8569 - val_accuracy: 0.4000
Epoch 7/10
1/1 [==============================] - 0s 68ms/step - loss: 186.8569 - accuracy: 0.4000 - val_loss: 118.7767 -

In [49]:
# Prediction example


pred = model.predict( np.expand_dims(X_train[0], axis=0) )
print(f"Pred {pred}: Real Label {y_train[0]}")

(2, 50)
(1, 2, 50)
Pred [[1. 0. 0. 0. 0.]]: Real Label 0.0


In [ ]:
ktfuhgdkjfhg
sdrser9